In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
data = pd.read_csv('movie/ratings.csv')

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   userId     1048575 non-null  int64  
 1   movieId    1048575 non-null  int64  
 2   rating     1048575 non-null  float64
 3   timestamp  1048575 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 32.0 MB


In [5]:
data_train = data.pivot(index = 'userId', columns = 'movieId', values = 'rating')

In [6]:
data_train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data_train.shape

(7120, 14026)

In [8]:
matrax = data_train.fillna(0)

In [9]:
matrax.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
matrix = np.array(matrax)

In [11]:
matrix.shape

(7120, 14026)

In [12]:
data = torch.tensor(matrix)

In [13]:
data.shape

torch.Size([7120, 14026])

In [14]:
u, s, v = torch.svd(data)

In [15]:
u.shape

torch.Size([7120, 7120])

In [16]:
s.shape

torch.Size([7120])

In [17]:
v.shape

torch.Size([14026, 7120])

In [18]:
s_topk = torch.topk(s, 100)
idx_list = s_topk[1].tolist()
s_topk[1].squeeze()

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [19]:
u_topk = torch.index_select(u, 1, s_topk[1].squeeze())
u_topk.shape

torch.Size([7120, 100])

In [22]:
v_topk = torch.index_select(v, 1, s_topk[1].squeeze())

In [26]:
v_topk.t().shape

torch.Size([100, 14026])

In [29]:
matrix_data = torch.mm(u_topk, v_topk.t())

In [31]:
matrix_data.shape

torch.Size([7120, 14026])

In [32]:
s

tensor([1.7097e+03, 6.7319e+02, 6.3651e+02,  ..., 1.5091e-13, 1.5091e-13,
        1.5091e-13], dtype=torch.float64)

利用ALS算法

In [14]:
data.shape[1]

14026

In [15]:
device = torch.device('cuda', 0)

In [16]:
data = data.to(device)
data.device

device(type='cuda', index=0)

In [17]:
def funk_svd(data, k, u=None, v=None):
    if u is None and v is None:
        u = torch.normal(mean=0, std=0.1,size=(data.shape[0], k), device = device, requires_grad=True)
        v = torch.normal(mean=0, std=0.1,size=(k, data.shape[1]), device = device, requires_grad=True)
    y_hat = torch.mm(u, v).to(device)
    return y_hat, u, v

In [18]:
def MSE_loss(y, y_hat):
    loss = ((y_hat - y)**2).sum().to(device)
    return loss

In [19]:
def Als(params, lr):
    for param in params:
        param.data.sub_(lr * param.grad)
        param.grad.data.zero_()

In [20]:
lr = 0.01
epochs = 3
net = funk_svd
loss = MSE_loss
u, v = None, None

In [23]:
for epoch in range(epochs):
    data_hat, u, v = net(data, 100, u, v)
    l = loss(data, data_hat)
    print(u.grad)
    Als([u, v], lr)

    print('epoch:{}, loss:{}'.format(epoch, l)) 
    

RuntimeError: CUDA out of memory. Tried to allocate 762.00 MiB (GPU 0; 4.00 GiB total capacity; 2.98 GiB already allocated; 112.76 MiB free; 3.00 GiB reserved in total by PyTorch)

In [17]:
a = torch.tensor([[1, 2, 3],[4, 5, 6]])
k = (a > 2).nonzero()

k

tensor([[0, 2],
        [1, 0],
        [1, 1],
        [1, 2]])

In [25]:
y = torch.tensor([1, 2, 3, 4, 0, 5])
one = torch.ones_like(y)
record = torch.where(y!=0,y,one)
y.mul(record)

tensor([ 1,  4,  9, 16,  0, 25])